Profiling results (easier to just look in the folder and see the txt files. But since we have to do this, here you go:)

File: C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot.py
Function: mandelbrot_point at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           @profile
    14                                           def mandelbrot_point(c: complex, max_iter: int = 100) -> int:
    15                                               """
    16                                               Compute the escape iteration count for a single complex number c.
    17
    18                                               We iterate:
    19                                               z_{n+1} = z_n^2 + c
    20                                               starting from z_0 = 0.
    21
    22                                               If |z| exceeds 2, the sequence will diverge, and we return the
    23                                               iteration number where that happens. If it never escapes within
    24                                               max_iter iterations, we return max_iter.
    25
    26                                               Parameters:
    27                                               c: complex
    28                                               Point in the complex plane.
    29
    30                                               max_iter: int
    31                                               Maximum number of iterations.
    32
    33                                               Returns:
    34                                               int:
    35                                               Number of iterations before escape, or max_iter if bounded.
    36                                               """
    37
    38    262144      45122.3      0.2      1.4      z = 0 + 0j  # start value
    39
    40   3259994     524204.7      0.2     15.7      for n in range(max_iter):
    41   3213910     865338.2      0.3     26.0          z = z * z + c
    42
    43                                                   # Check escape condition |z| > 2 squared
    44   3213910    1859934.8      0.6     55.8          if (z.real * z.real + z.imag * z.imag) > 4:    %%% bottleneck found (police siren sound)
    45    216060      32814.8      0.2      1.0              return n
    46
    47     46084       6035.0      0.1      0.2      return max_iter


==== naive_profile.prof ====
Thu Feb 26 13:27:13 2026    naive_profile.prof

         262176 function calls in 1.874 seconds

   Ordered by: cumulative time
   List reduced from 20 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.874    1.874 {built-in method builtins.exec}
        1    0.000    0.000    1.874    1.874 <string>:1(<module>)
        1    0.000    0.000    1.874    1.874 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot.py:127(mandelbrot_naive)
        1    0.279    0.279    1.874    1.874 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot.py:52(compute_mandelbrot_grid)
   262144    1.595    0.000    1.595    0.000 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot.py:13(mandelbrot_point)
        2    0.000    0.000    0.000    0.000 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\_core\function_base.py:26(linspace)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.arange}
        2    0.000    0.000    0.000    0.000 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\_core\numeric.py:1964(isscalar)
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}



==== numpy_profile.prof ====
Thu Feb 26 13:27:13 2026    numpy_profile.prof

         79 function calls in 0.241 seconds

   Ordered by: cumulative time
   List reduced from 44 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.241    0.241 {built-in method builtins.exec}
        1    0.000    0.000    0.241    0.241 <string>:1(<module>)
        1    0.001    0.001    0.241    0.241 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot_np.py:68(mandelbrot_numpy)
        1    0.232    0.232    0.234    0.234 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot_np.py:16(mandelbrot_vectorized)
        1    0.004    0.004    0.006    0.006 C:\Users\benja\OneDrive\Documents\GitHub\Numeral-Scientific-Computing\mandelbrot\mandelbrot_np.py:6(make_complex_grid)
        1    0.000    0.000    0.002    0.002 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\lib\_function_base_impl.py:5134(meshgrid)
        1    0.002    0.002    0.002    0.002 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\_core\numeric.py:98(zeros_like)
        3    0.000    0.000    0.001    0.000 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\lib\_function_base_impl.py:5280(<genexpr>)
        2    0.001    0.001    0.001    0.001 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.001    0.001 C:\Users\benja\miniforge3\envs\nsc2026\Lib\site-packages\numpy\lib\_stride_tricks_impl.py:481(broadcast_arrays)

In [ ]:
Questions: 1:  cProfile on naive vs NumPy: How many functions appear in each profile? What does this difference tell you about where the work actually happens?
Naive: 20 functions show up (262,176 calls total).
NumPy: 44 functions show up (79 calls total), but most time is basically in one call into NumPy.
This means that in the naive version the work is happening in Python code and the NumPy version the work happens mostly in compiled NumPy internals, 
so Python only makes a handful of calls and then hands off the heavy lifting to NymPy.


2: line profiler on naive: Which lines dominate runtime? What fraction of total time is spent in the inner loop?
The runtime is dominated by:
if (z.real*z.real + z.imag*z.imag) > 4:   %This line is 55.8%
z = z*z + c     %This line is 26%

the "for n in range(max_iter):" loop overhead itself is 15.7%
So basically the inner loop is 97.5% of the time


3: Based on your profiling results: why is NumPy faster than naive Python?
Because it avoids doing millions of Python-level iterations. The naive version spends almost all its time stepping z = z*z + c and checking escape in Python. 
NumPy pushes that work into C loops so the interpreter overhead mostly disappears.


4: What would you need to change to make the naive version faster? (hint: what does line profiler tell you about the inner loop?)
We need to get rid of the Python inner loop cost. The line profiler makes it obvious: the hot spot is the per-iteration update + escape check. 
So two options:
compile that loop (Numba on mandelbrot_point / whole grid loop) or rewrite it so the iteration happens in vectorized / NumPY.

In [ ]:
Speedup Table:

     | Time |
Type | 
_____|_______
Naive|  4,877
_____|_______
Numpy| 0,68s
_____|_______
Numba| 0.7s  (hybrid)
Numba| 0.006s (fully)


In [ ]:
Data type comparison: Mandelbrot Numba dtype test (fully compiled-ish)
grid: 1024x1024, max_iter: 50
float16 (emulated as float32): 0.008s    %We just fake it lmao
float32: 0.007s
float64: 0.007s

Looking at the visual, imma be real: Cant se shit for difference. See the numba_float_visual.png
